In [1]:
import os, time, re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from scipy.special import softmax
import joblib
from joblib import Parallel, delayed
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import  RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.base import BaseEstimator, TransformerMixin

### This loads all of the LOFO ranked subsets and trains 499 models, one for each subset.

In [6]:
# Points to the training data.
train_dir = '../data/ranked_subsets/train_features_csv'

# Creates subset CSV list.
csv_files = [f'top_{i}_features.csv' for i in range(1, 500)]

# Init dictionary to store dataframes.
dfs = {}

# Iteratively load CSVs.
for feature_number in tqdm(range(1, 500), desc="Loading CSV files"):
    df_name = f"df{feature_number}"
    file_path = os.path.join(train_dir, f'top_{feature_number}_features.csv')
    df = pd.read_csv(file_path, low_memory=False)
    dfs[df_name] = df

print(f"Loaded {len(dfs)} dataframes.")
print(dfs.keys())

Loading CSV files:   0%|          | 0/499 [00:00<?, ?it/s]

Loaded 499 dataframes.
dict_keys(['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df9', 'df10', 'df11', 'df12', 'df13', 'df14', 'df15', 'df16', 'df17', 'df18', 'df19', 'df20', 'df21', 'df22', 'df23', 'df24', 'df25', 'df26', 'df27', 'df28', 'df29', 'df30', 'df31', 'df32', 'df33', 'df34', 'df35', 'df36', 'df37', 'df38', 'df39', 'df40', 'df41', 'df42', 'df43', 'df44', 'df45', 'df46', 'df47', 'df48', 'df49', 'df50', 'df51', 'df52', 'df53', 'df54', 'df55', 'df56', 'df57', 'df58', 'df59', 'df60', 'df61', 'df62', 'df63', 'df64', 'df65', 'df66', 'df67', 'df68', 'df69', 'df70', 'df71', 'df72', 'df73', 'df74', 'df75', 'df76', 'df77', 'df78', 'df79', 'df80', 'df81', 'df82', 'df83', 'df84', 'df85', 'df86', 'df87', 'df88', 'df89', 'df90', 'df91', 'df92', 'df93', 'df94', 'df95', 'df96', 'df97', 'df98', 'df99', 'df100', 'df101', 'df102', 'df103', 'df104', 'df105', 'df106', 'df107', 'df108', 'df109', 'df110', 'df111', 'df112', 'df113', 'df114', 'df115', 'df116', 'df117', 'df118', 'df119', 'df

In [2]:
# These are the best params for XGBoost.
import json
with open('models/top_27_params.json', 'r') as f:
    top_n_params = json.load(f)

In [3]:
# Load the encoded NEGONE and MEDIAN imputation lists.
file_path = '../data/negone_median.json'

with open(file_path, 'r') as file:
    data = json.load(file)

negone_features = data.get('NEGONE_FEATURES', [])
median_features = data.get('MEDIAN_FEATURES', [])

print('NEGONE_FEATURES:', negone_features)
print('MEDIAN_FEATURES:', median_features)

NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature46', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature51', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94', 'feature95', 'feature96', 'feature97', 'feature98', 'feature99', 'feature100', 'feature101', 'feature102', 'feature103', 'feature105', 'feature107', 'fea

In [9]:
# Load the encoded features used by the pre-processing pipeline.
file_path = '../data/patterns.json'

# Load JSON.
with open(file_path, 'r') as file:
    data = json.load(file)

# Pull keys out and turn into integer numbers.
keys_list = [int(key.replace('feature', '')) for key in data.keys()]
drop_numbers = keys_list
print('Extracted Keys:', drop_numbers)
print('Length:', len(drop_numbers))

Extracted Keys: [35, 36, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]
Length: 26


In [22]:
""" Iterate through the entire training script for each subeset DataFrame in the dictionary. """

train_dfs = dfs
for df_name, X_train in train_dfs.items():
    y_train = pd.read_csv('../data/y_train_id.csv')
    print(y_train.shape)

    # Encoding the labels.
    y_train_enc = []
    for lab in y_train['label']:
        if lab == 'GOF':
            y_train_enc.append(1)
        elif lab == 'LOF':
            y_train_enc.append(2)
        else:
            y_train_enc.append(0)
    y_train = y_train_enc
    print(len(y_train))

    print(f"Processing {df_name}")

    model_name = f"xgb_{df_name}"
    model_directory = f'../more-features/models/{model_name}'

    # Create the directory.
    if not os.path.exists(model_directory):
        os.makedirs(model_directory)

    print(f"Directory '{model_directory}' is ready.")

    # These lists are used to determine imputation strategies for the specified features (inherited from LoGoFunc)
    NEGONE_FEATURES = ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature46', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature51', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94', 'feature95', 'feature96', 'feature97', 'feature98', 'feature99', 'feature100', 'feature101', 'feature102', 'feature103', 'feature105', 'feature107', 'feature108', 'feature109', 'feature110', 'feature111', 'feature112', 'feature113', 'feature114', 'feature115', 'feature116', 'feature117', 'feature118', 'feature119', 'feature120', 'feature121', 'feature122', 'feature123', 'feature124', 'feature125', 'feature126', 'feature127', 'feature128', 'feature129', 'feature130', 'feature131', 'feature132', 'feature133', 'feature134', 'feature135', 'feature136', 'feature137', 'feature138', 'feature139', 'feature140', 'feature141', 'feature142', 'feature143', 'feature144', 'feature145', 'feature146', 'feature147', 'feature148', 'feature149', 'feature150', 'feature151', 'feature152', 'feature153', 'feature154', 'feature155', 'feature156', 'feature157', 'feature158', 'feature159', 'feature160', 'feature161', 'feature162', 'feature163', 'feature164', 'feature165', 'feature166', 'feature167', 'feature168', 'feature169', 'feature170', 'feature171', 'feature172', 'feature173', 'feature174', 'feature175', 'feature176', 'feature177', 'feature178', 'feature179', 'feature180', 'feature181', 'feature182', 'feature183', 'feature184', 'feature185', 'feature186', 'feature187', 'feature188', 'feature189', 'feature190', 'feature191', 'feature192', 'feature193', 'feature194', 'feature195', 'feature196', 'feature197', 'feature198', 'feature199', 'feature200', 'feature201', 'feature202', 'feature203', 'feature204', 'feature205', 'feature206', 'feature207', 'feature208', 'feature209', 'feature210', 'feature211', 'feature212', 'feature213', 'feature214', 'feature215', 'feature216', 'feature217', 'feature218', 'feature219', 'feature220', 'feature221', 'feature222', 'feature223', 'feature224', 'feature225', 'feature226', 'feature227', 'feature228', 'feature229', 'feature230', 'feature231', 'feature232', 'feature233', 'feature234', 'feature235', 'feature236', 'feature237', 'feature238', 'feature239', 'feature240', 'feature241', 'feature242', 'feature243', 'feature244', 'feature245', 'feature246', 'feature247', 'feature248', 'feature249', 'feature250', 'feature251', 'feature252', 'feature253', 'feature254', 'feature255', 'feature256', 'feature257', 'feature258', 'feature259', 'feature260', 'feature261', 'feature262', 'feature263', 'feature264', 'feature265', 'feature266', 'feature267', 'feature268', 'feature269', 'feature270', 'feature271', 'feature272', 'feature273', 'feature274', 'feature275', 'feature276', 'feature277', 'feature278', 'feature279', 'feature280', 'feature281', 'feature282', 'feature283', 'feature284', 'feature285', 'feature286', 'feature287', 'feature288', 'feature289', 'feature290', 'feature291', 'feature292', 'feature293', 'feature294', 'feature295', 'feature296', 'feature297', 'feature298', 'feature299', 'feature300', 'feature301', 'feature302', 'feature303', 'feature304', 'feature305', 'feature306', 'feature307', 'feature308', 'feature309', 'feature310', 'feature311', 'feature312', 'feature313', 'feature314', 'feature315', 'feature316', 'feature317', 'feature318', 'feature319', 'feature320', 'feature321', 'feature322', 'feature323', 'feature324', 'feature325', 'feature326', 'feature327', 'feature328', 'feature329', 'feature330', 'feature331', 'feature332', 'feature333', 'feature334', 'feature335', 'feature336', 'feature337', 'feature338', 'feature339', 'feature340', 'feature341', 'feature342', 'feature343', 'feature344', 'feature345', 'feature346', 'feature347', 'feature348', 'feature349', 'feature355', 'feature356', 'feature357', 'feature358', 'feature359', 'feature360', 'feature361', 'feature362', 'feature363', 'feature364', 'feature365', 'feature366', 'feature367', 'feature368', 'feature369', 'feature373', 'feature374', 'feature375', 'feature376', 'feature421', 'feature422', 'feature423', 'feature424', 'feature425', 'feature426', 'feature427', 'feature428', 'feature429', 'feature430', 'feature431', 'feature432', 'feature433', 'feature434', 'feature435', 'feature436', 'feature437', 'feature438', 'feature439', 'feature440', 'feature441', 'feature442', 'feature443', 'feature444', 'feature445', 'feature446', 'feature447', 'feature448', 'feature459', 'feature460', 'feature461', 'feature462', 'feature463', 'feature464', 'feature465', 'feature466', 'feature467', 'feature471', 'feature472', 'feature473', 'feature474', 'feature475', 'feature476', 'feature477', 'feature478', 'feature479', 'feature480', 'feature481', 'feature482', 'feature483', 'feature484', 'feature485', 'feature486']
    MEDIAN_FEATURES = ['feature3', 'feature5', 'feature7', 'feature8', 'feature9', 'feature11', 'feature12', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'feature20', 'feature21', 'feature22', 'feature23', 'feature24', 'feature25', 'feature26', 'feature27', 'feature28', 'feature29', 'feature30', 'feature32', 'feature33', 'feature34', 'feature37', 'feature38', 'feature39', 'feature40', 'feature41', 'feature42', 'feature43', 'feature79', 'feature80', 'feature81', 'feature82', 'feature350', 'feature351', 'feature352', 'feature353', 'feature354', 'feature370', 'feature371', 'feature372', 'feature378', 'feature379', 'feature380', 'feature381', 'feature382', 'feature383', 'feature384', 'feature385', 'feature386', 'feature387', 'feature388', 'feature389', 'feature390', 'feature391', 'feature392', 'feature393', 'feature394', 'feature395', 'feature396', 'feature397', 'feature398', 'feature399', 'feature400', 'feature401', 'feature402', 'feature403', 'feature404', 'feature405', 'feature406', 'feature407', 'feature408', 'feature409', 'feature410', 'feature411', 'feature412', 'feature413', 'feature414', 'feature415', 'feature416', 'feature417', 'feature418', 'feature419', 'feature420', 'feature449', 'feature450', 'feature451', 'feature452', 'feature453', 'feature454', 'feature455', 'feature456', 'feature457', 'feature469', 'feature470']

    # Filter NEGONE_FEATURES and MEDIAN_FEATURES based on current X_train columns.
    negone_features_filtered = [feature for feature in NEGONE_FEATURES if feature in X_train.columns]
    median_features_filtered = [feature for feature in MEDIAN_FEATURES if feature in X_train.columns]

    # Redefine NEGONE_FEATURES and MEDIAN_FEATURES with updated values.
    NEGONE_FEATURES = negone_features_filtered
    MEDIAN_FEATURES = median_features_filtered

    print("Filtered NEGONE_FEATURES:", NEGONE_FEATURES)
    print("Filtered MEDIAN_FEATURES:", MEDIAN_FEATURES)

    def generate_preprocessor(numeric_features, categorical_features, N_JOBS, cat_encode_type, 
                                do_specificimpute, do_featureselection, 
                                do_sampling, do_pca, var_thresh, oversample_technique, 
                                negone_features=NEGONE_FEATURES, median_features=MEDIAN_FEATURES,
                                prefix='', do_feature_subset=False, max_features=1, do_removeppi=False, do_removegtex=False):
        cat_encoders = [OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1), 
                        OneHotEncoder(sparse=False, handle_unknown='infrequent_if_exist', min_frequency=10)]
        categorical_transformer = cat_encoders[cat_encode_type]

        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', MinMaxScaler(feature_range =(0, 1), clip=True))])

        median_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', MinMaxScaler(feature_range =(0, 1), clip=True))])

        negone_transformer = Pipeline(steps=[
            ('scaler', MinMaxScaler(feature_range =(0, 1), clip=True)),
            ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
        ])

        preprocessor = None
        if do_specificimpute:
            preprocessor = ColumnTransformer(
                transformers=[
                    ('median', median_transformer, median_features),
                    ('negone', negone_transformer, negone_features),
                    ('cat', categorical_transformer, categorical_features),
            ])
        else:
            preprocessor = ColumnTransformer(
                transformers=[
                    ('numeric', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features),
                ])

        vt = VarianceThreshold(threshold=var_thresh)
        steps = [('initial', preprocessor), ('removeba', RemoveBeforeAfterTransformer()), ('variance_threshold', vt)]
        if do_sampling == 1:
            steps.append(('undersampling', RandomUnderSampler(random_state=42)))
        if do_sampling == 2:
            oversamplers = [SMOTE(n_jobs=N_JOBS,random_state=42), RandomOverSampler(random_state=42)]
            steps.append(('oversampling', oversamplers[oversample_technique]))
        if do_pca:
            steps.append(('pca', PCA()))

        preprocessor = Pipeline(steps=steps)
        return preprocessor

    class RemoveBeforeAfterTransformer(BaseEstimator, TransformerMixin):
        def __init__(self):
            self.drop_cols = None

        def fit(self, X, y=None):
            print(X.shape)
            self.drop_cols = [col for col in X.columns if any(f'feature{num}' == col for num in drop_numbers)]
            print(f"Columns to be dropped: {self.drop_cols}")
            return self

        def transform(self, X, y=None):
            print(f"Columns being dropped: {[col for col in self.drop_cols if col in X.columns]}")
            X = X.drop(columns=self.drop_cols, errors='ignore')
            print(X.shape)
            return X

        def get_feature_names_out(self, input_features=None):
            return [f for f in input_features if f not in self.drop_cols]

    def preprocess(preprocessor, train_data, train_labels, quiet=False):
        for k, v in preprocessor.steps:
            print(f"Applying step: {k}")
            print(f"Input shape: {train_data.shape}")
            print(f"Input columns: {train_data.columns}")
            
            if k == 'initial':
                start = time.time()
                v.fit(train_data)
                train_data = pd.DataFrame(v.transform(train_data), columns=v.get_feature_names_out())
                end = time.time()
                if not quiet:
                    print(k + ' took ' + str(end - start) + ' to run.')
            elif k == 'oversampling' or k == 'undersampling':
                start = time.time()
                input_features = train_data.columns
                train_data, train_labels = v.fit_resample(train_data, train_labels)
                train_data = pd.DataFrame(train_data, columns=input_features)
                end = time.time()
                if not quiet:
                    print(k + ' took ' + str(end - start) + ' to run.')
            else:
                start = time.time()
                v.fit(train_data)
                input_features = train_data.columns
                train_data = pd.DataFrame(v.transform(train_data), columns=v.get_feature_names_out(input_features))
                end = time.time()
                if not quiet:
                    print(k + ' took ' + str(end - start) + ' to run.')
            
            print(f"Output shape: {train_data.shape}")
            print(f"Output columns: {train_data.columns}")
            print("---")

        for col in train_data.columns:
            try:
                train_data[col] = train_data[col].astype('float')
            except:
                train_data[col] = train_data[col].astype('category')

        return train_data, train_labels

    def transform(test_data, preprocessor, quiet=False):
        for k, v in preprocessor.steps:
            if k == 'initial':
                test_data = pd.DataFrame(v.transform(test_data), columns=v.get_feature_names_out())
            elif k == 'oversampling' or k == 'undersampling':
                continue
            else:
                test_data = v.transform(test_data)
        test_data = pd.DataFrame(test_data)

        for col in test_data.columns:
            try:
                test_data[col] = test_data[col].astype('float')
            except:
                test_data[col] = test_data[col].astype('category')

        return test_data.to_numpy()

    # Specify the paths for model, results, and metrics.
    model_path = model_directory
    output_path = f'results/{model_name}-validation.csv'
    metrics_file = f'metrics/{model_name}-validation-metrics.csv'

    # This is used to drop columns that only contain NaN values.
    def drop_allnan(data):
        for col in data.columns:
            if data[col].isna().sum() == len(data):
                data = data.drop(columns=col)
        return data

    # This encodes the IMPACT feature's column.
    X_train = drop_allnan(X_train)
    feature1_vals = {'LOW': 0, 'MODIFIER': 1, 'MODERATE': 1.5, 'HIGH': 2}

    for col in X_train.columns:
        if X_train[col].dtype == 'object':
            if set(X_train[col].unique()) <= set(feature1_vals.keys()):
                X_train[col] = X_train[col].map(feature1_vals).fillna(0)

    # Conditional imputation based on feature type.
    numeric_features = X_train.select_dtypes(include=['number']).columns
    categorical_features = X_train.select_dtypes(include=['object']).columns

    # Pre-processor generation with multiple parameters that can be tweaked. I used the same values as LoGoFunc
    preprocessor = generate_preprocessor(numeric_features, categorical_features, 40, 0,
                                1, 0, 2, 0, 0, 1,
                                prefix='light0', do_feature_subset=True, max_features=1)

    if isinstance(y_train, list):
        y_train = pd.DataFrame({'label': y_train})

    X_train, y_train = preprocess(preprocessor, X_train, y_train)

    # This creates a directory for the preprocessor and trained models.
    joblib.dump(preprocessor, f'{model_path}/preprocessor.joblib')
    os.makedirs(model_path, exist_ok=True)

    def train_model(params, X_train, y_train):
        model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss', **params)
        model.fit(X_train, y_train)
        return model

    # Train the ensemble in parallel.
    models = Parallel(n_jobs=-1)(delayed(train_model)(params, X_train, y_train) for params in top_n_params)

    # Export models and params.
    for i, (model, params) in enumerate(zip(models, top_n_params)):
        joblib.dump(model, f'{model_path}/xgb_model_{i}.joblib')
        with open(f'{model_path}/xgb_params_{i}.json', 'w') as f:
            json.dump(params, f, indent=2)

    print(f"Training complete for {df_name}. {len(models)} models have been trained and saved.")

print("All training completed.")

(25546, 1)
25546
Processing df367
Directory '../more-features/models/xgb_df367' is ready.
Filtered NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94', 'feature95', 'feature96', 'feature97', 'feature98', 'feature99', 'feature100', '

/Users/benzenesea/mambaforge/envs/logofunc3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Training complete for df368. 27 models have been trained and saved.
(25546, 1)
25546
Processing df369
Directory '../more-features/models/xgb_df369' is ready.
Filtered NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94', 'feature95',

/Users/benzenesea/mambaforge/envs/logofunc3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Training complete for df369. 27 models have been trained and saved.
(25546, 1)
25546
Processing df370
Directory '../more-features/models/xgb_df370' is ready.
Filtered NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94', 'feature95',

...

Training complete for df487. 27 models have been trained and saved.
(25546, 1)
25546
Processing df488
Directory '../more-features/models/xgb_df488' is ready.
Filtered NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature51', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94',

/Users/benzenesea/mambaforge/envs/logofunc3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Training complete for df495. 27 models have been trained and saved.
(25546, 1)
25546
Processing df496
Directory '../more-features/models/xgb_df496' is ready.
Filtered NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature51', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93', 'feature94',

/Users/benzenesea/mambaforge/envs/logofunc3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Training complete for df497. 27 models have been trained and saved.
(25546, 1)
25546
Processing df498
Directory '../more-features/models/xgb_df498' is ready.
Filtered NEGONE_FEATURES: ['feature487', 'feature489', 'feature491', 'feature492', 'feature493', 'feature494', 'feature495', 'feature496', 'feature497', 'feature498', 'feature499', 'feature458', 'feature4', 'feature6', 'feature10', 'feature13', 'feature31', 'feature35', 'feature36', 'feature46', 'feature47', 'feature48', 'feature49', 'feature45', 'feature50', 'feature51', 'feature52', 'feature53', 'feature54', 'feature55', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature62', 'feature63', 'feature64', 'feature65', 'feature66', 'feature67', 'feature68', 'feature69', 'feature70', 'feature71', 'feature72', 'feature73', 'feature74', 'feature75', 'feature76', 'feature77', 'feature84', 'feature85', 'feature86', 'feature87', 'feature88', 'feature89', 'feature90', 'feature91', 'feature92', 'feature93',